# Dynamic plots (Python)

In [57]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# import dash_table_experiments as dt

import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.plotly as py
import plotly.tools as pytools
import flask
import os

import pandas as pd
import numpy as np
from textwrap import dedent as s

In [74]:
# get data
df = pd.read_csv('../dash/data/shifts_pred.csv')

In [16]:
#df.head(10)

In [59]:
import plotly.figure_factory as ff

In [60]:
countries = df.ctry.unique()
label_countries = [w.replace('_', ' ') for w in countries]
dict_countries = dict(zip(countries, label_countries))

In [61]:
df.head()

In [64]:
df.loc[df.ctry=='Argentina', 'segment'].value_counts()

<1950        16000
1970-1989    12000
1950-1969    12000
>=1990        8000
Name: segment, dtype: int64

In [76]:
hist_data = []
for i in df.loc[(df.ctry=='Argentina') & (df.year==1950), 'segment'].unique(): 
    hist_data.append(df.loc[(df.ctry=='Argentina') & (df.year==1950) & (df.segment==i), 
                              'pred_shift'].values)

In [78]:
group_labels = ['<1950', '1970-1989', '>=1990']
colors = ['#e34a33', '#2b8cbe', '#31a354']


fig = ff.create_distplot(hist_data, group_labels=group_labels,
                         curve_type='kde', show_hist=False, show_rug=False, 
                         colors=colors)

fig['layout'].update(legend=dict(orientation='h', x=0.32, y=1.1, traceorder='normal'))
fig['layout'].update(xaxis=dict(range=[np.concatenate(hist_data).min(), np.concatenate(hist_data).max()], ))
# fig['layout'].update(yaxis=dict(range=[0,1]))

# Add title
fig['layout'].update(title='Expected Shift X in Year Y by Year Segment')

# Plot!
py.iplot(fig, filename='shift curves')

# Prediction checks

In [71]:
pred = pd.read_csv('../dash/data/pred.csv')

In [72]:
dt = pred.loc[pred.ctry=='Argentina']

upper_bound = go.Scatter(
    name='Upper Bound',
    x=dt['year'],
    y=dt['hi'],
    mode='lines',
    marker=dict(color="444"),
    line=dict(width=0),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty')

points = go.Scatter(
    name='Observed LE',
    x=dt['year'],
    y=dt['le'],
    mode='markers', 
    marker=dict(opacity=0.5))

trace = go.Scatter(
    name='Predicted LE',
    x=dt['year'],
    y=dt['m'],
    mode='lines',
    line=dict(color='rgb(31, 119, 180)'),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty')

lower_bound = go.Scatter(
    name='Lower Bound',
    x=dt['year'],
    y=dt['lo'],
    marker=dict(color="444"),
    line=dict(width=0),
    mode='lines')

# Trace order can be important
# with continuous error bars
data = [lower_bound, trace, upper_bound, points]

layout = go.Layout(
    yaxis=dict(title='LE'),
    xaxis=dict(title='Year'),
    title='Observed vs Predicted LE',
    showlegend = False)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='prediction checks')